In [1]:
from Dataloader import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pkuseg
import gensim

In [2]:
# 加载数据
loader = Dataloader('../启明学院智能客服知识库.xlsx', '知识库')

In [3]:
#处理加载的数据，建立索引，去重
sentince_list = []
sentince_index = []
for st in loader._data_pool:
    sentince_list.append(st[0])
    sentince_list.append(st[1])
    sentince_index.append((st[0],st[2]))
    sentince_index.append((st[1],st[2]))
sentince_list = list(set(sentince_list))
sentince_index = list(set(sentince_index))
sentince_index = {each[0]:each[1] for each in sentince_index}

In [4]:
#使用TF-IDF将句子转化为向量形式,形成矩阵
seg = pkuseg.pkuseg()           # 以默认配置加载模型
tfidf_vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", max_df=0.7)

real_documents = []
for item_text in sentince_list:
    item_str = seg.cut(item_text)
    real_documents.append(item_str)
document = [" ".join(sent0) for sent0 in real_documents]

tfidf_model = tfidf_vectorizer.fit(document)
#print(tfidf_model.vocabulary_)

#print(tfidf_vectorizer.idf_) #特征对应的权重
#print(tfidf_vectorizer.get_feature_names())#特征词
#print(real_vec.toarray()) #上面对话对应的向量表示
sparse_result = tfidf_model.transform(document).todense()

In [10]:
#测试句子转换
test_sentices = ['启明学院是什么']
test_documents = []
for item_text in test_sentices:
    item_str = seg.cut(item_text)
    test_documents.append(item_str)
test_document = [" ".join(sent0) for sent0 in test_documents]
result = tfidf_model.transform(test_document).todense()


In [11]:
#结果测试
ans = result*sparse_result.T
ans = ans.argmax(axis=1)
answer = sentince_list[ans.tolist()[0][0]]
answer = loader.get_answer(sentince_index[answer])
print(answer)

启明学院秉承"明德，厚学，求是，创新"的校训，遵循"育人为本、创新是魂、责任以行"的办学理念，把创新精神和实践能力作为培养拔尖创新人才的核心，致力于培养具有创新能力、创业精神和国际视野的未来杰出人才。
启明学院基本办学思路为"开放办学，创新机制，学科交叉，拔尖示范"。将以"开放式"办学的思路统领基本工作，以先进的教育理念为指导，建立有效的教育管理（包括选拔、评价、激励、约束等）机制，创立新的运行机制和多样化拔尖创新人才培养模式。围绕拔尖创新人才的培养目标，构建多学科交叉平台，加强学科间的交叉渗透和跨学科的合作与研究，为培养创新、创业型人才提供学科支撑。面向本科生，突出学生创新能力、创业精神和领导能力的培养，特别是实学创新人才的培养，以此推动面上的创新教育，为我国高等教育改革提供示范。
具体详细介绍见学校官网
